In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.69,-122.33,SEATTLE,NR3,440000,327000,1926,3000,1100,7,7,3,1.50,3,2,0,0,370,0,0,0,0,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.60,-122.29,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.63,-122.31,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.68,-122.39,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [ ]:
df.describe()

,id,sale_price,latitude,longitude,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
count,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00,24938.00
mean,12474.65,1064284.58,47.63,-122.34,487012.55,487310.74,1942.58,6001.87,1902.27,7.32,4.09,3.73,1.35,3.24,1.40,0.52,0.29,83.83,63.44,0.03,0.00,0.00,0.26,0.02,0.12,0.11,0.40,0.04,0.13,0.10,0.02,2020.05
std,7202.83,660373.89,0.07,0.04,344108.94,347250.79,27.60,3656.75,852.55,1.04,3.41,0.80,0.46,0.99,0.65,0.66,0.48,159.81,149.03,0.44,0.04,0.02,0.62,0.24,0.56,0.53,0.95,0.32,0.57,0.51,0.20,1.36
min,0.00,65000.00,47.50,-122.42,0.00,0.00,1900.00,773.00,260.00,3.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2018.00
25%,6236.25,699000.00,47.56,-122.38,291000.00,279000.00,1922.00,4200.00,1290.00,7.00,0.00,3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2019.00
50%,12476.50,890000.00,47.64,-122.34,432000.00,427000.00,1941.00,5355.00,1780.00,7.00,6.00,4.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2020.00
75%,18711.75,1250000.00,47.68,-122.30,605000.00,606000.00,1953.00,6960.00,2340.00,8.00,7.00,4.00,1.50,4.00,2.00,1.00,1.00,180.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2021.00
max,24949.00,14850000.00,47.73,-122.24,7560000.00,7946000.00,2022.00,155267.00,12280.00,13.00,13.00,5.00,4.00,9.00,5.00,5.00,5.00,1860.00,1520.00,8.00,1.00,1.00,3.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,2022.00


In [ ]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_month'] = df['sale_date'].dt.month
df['sale_day'] = df['sale_date'].dt.day
df.rename(columns={'year': 'sale_year'}, inplace=True)
df.head()

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
3,3,2018-02-13,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
4,4,2018-02-14,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14


In [ ]:
df['garb_sqft'].value_counts()

0       18360
200      1084
240       607
220       405
180       339
        ...  
1170        1
950         1
115         1
1370        1
255         1
Name: garb_sqft, Length: 119, dtype: int64

In [ ]:
df['gara_sqft'].value_counts()

0       20197
240       302
400       246
440       232
220       227
        ...  
1480        1
900         1
1180        1
880         1
1050        1
Name: gara_sqft, Length: 123, dtype: int64

In [ ]:
df['wfnt'].value_counts()

0    24775
6       99
3       48
8       11
5        3
4        2
Name: wfnt, dtype: int64

In [ ]:
df = df.drop(columns=['wfnt'])
df = df.drop(columns=['garb_sqft'])
df = df.drop(columns=['gara_sqft'])
df = df.drop(columns=['id'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24938 entries, 0 to 24937
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   sale_date         24938 non-null  datetime64[ns]
 1   sale_price        24938 non-null  int64         
 2   latitude          24938 non-null  float64       
 3   longitude         24938 non-null  float64       
 4   city              24938 non-null  object        
 5   zoning            24938 non-null  object        
 6   land_val          24938 non-null  int64         
 7   imp_val           24938 non-null  int64         
 8   year_built        24938 non-null  int64         
 9   sqft_lot          24938 non-null  int64         
 10  sqft              24938 non-null  int64         
 11  grade             24938 non-null  int64         
 12  fbsmt_grade       24938 non-null  int64         
 13  home_condition    24938 non-null  int64         
 14  stories           2493

In [ ]:
df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
df['zoning'] = df['zoning'].map(category_mapping)
df['zoning'].value_counts()

1    16716
2     4014
3     2191
4     1293
5      360
6      313
7       51
Name: zoning, dtype: int64

In [ ]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_date'] = df['sale_date'].astype('int64') // 10**9
df['sale_date']

0        1514851200
1        1515456000
2        1518566400
3        1518480000
4        1518566400
            ...    
24933    1670457600
24934    1670198400
24935    1671235200
24936    1671062400
24937    1671148800
Name: sale_date, Length: 24938, dtype: int64

In [ ]:
df = df.drop(columns=['city'])

In [ ]:
df = df.drop(columns=['sale_date'])

In [ ]:
# Specify the column to exclude from the subset
column_to_exclude = 'sale_day'

# Create a subset of columns by excluding the specified column
subset_columns = [col for col in df.columns if col != column_to_exclude]

# Create a new DataFrame with the subset of columns
df_subset = df[subset_columns]

# Remove duplicate rows based on the subset of columns
df_no_duplicates = df_subset.drop_duplicates()
df = df_no_duplicates

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24818 entries, 0 to 24937
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sale_price        24818 non-null  int64  
 1   latitude          24818 non-null  float64
 2   longitude         24818 non-null  float64
 3   zoning            24818 non-null  int64  
 4   land_val          24818 non-null  int64  
 5   imp_val           24818 non-null  int64  
 6   year_built        24818 non-null  int64  
 7   sqft_lot          24818 non-null  int64  
 8   sqft              24818 non-null  int64  
 9   grade             24818 non-null  int64  
 10  fbsmt_grade       24818 non-null  int64  
 11  home_condition    24818 non-null  int64  
 12  stories           24818 non-null  float64
 13  beds              24818 non-null  int64  
 14  bath_full         24818 non-null  int64  
 15  bath_3qtr         24818 non-null  int64  
 16  bath_half         24818 non-null  int64 

In [ ]:
df.describe()

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
count,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00,24818.00
mean,1062638.23,47.63,-122.34,1.62,487062.73,487121.52,1942.57,6002.42,1901.74,7.32,4.09,3.73,1.35,3.24,1.40,0.52,0.29,0.00,0.00,0.26,0.02,0.12,0.11,0.40,0.04,0.13,0.10,0.02,2020.05,6.63
std,654882.86,0.07,0.04,1.10,344179.71,347188.46,27.57,3649.74,852.44,1.04,3.41,0.80,0.46,0.99,0.65,0.65,0.48,0.04,0.02,0.62,0.24,0.56,0.53,0.95,0.32,0.57,0.51,0.20,1.36,3.08
min,65000.00,47.50,-122.42,1.00,0.00,0.00,1900.00,773.00,260.00,3.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2018.00,1.00
25%,698000.00,47.56,-122.38,1.00,291000.00,279000.00,1922.00,4201.25,1290.00,7.00,0.00,3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2019.00,4.00
50%,889999.50,47.64,-122.34,1.00,432000.00,427000.00,1941.00,5357.50,1780.00,7.00,6.00,4.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2020.00,7.00
75%,1250000.00,47.68,-122.30,2.00,605000.00,606000.00,1953.00,6963.75,2340.00,8.00,7.00,4.00,1.50,4.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2021.00,9.00
max,14850000.00,47.73,-122.24,7.00,7560000.00,7946000.00,2022.00,155267.00,12280.00,13.00,13.00,5.00,4.00,9.00,5.00,5.00,5.00,1.00,1.00,3.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,2022.00,12.00


In [196]:
bin_df = df.copy()
bin_edges = [0, 1000000, 1500000, float('inf')]  # Define your own bin edges
bin_labels = [1, 2, 3]

# Apply binning to the 'sale_price' column
bin_df['price_category'] = pd.cut(bin_df['sale_price'], bins=bin_edges, labels=bin_labels)

# Display the DataFrame with the new 'price_category' column
print(bin_df.head())

   sale_price  latitude  longitude  zoning  land_val  imp_val  year_built  \
0      665000     47.56    -122.32       3     41000   122000        1920   
1      499000     47.52    -122.32       3     29000    55000        1907   
2      675000     47.62    -122.30       3     97000   132000        1908   
3     1150000     47.72    -122.34       5    440000   221000        1920   
4      705000     47.67    -122.32       3     81000    72000        1918   

   sqft_lot  sqft  grade  fbsmt_grade  home_condition  stories  beds  \
0      2525  1570      6            4               4     1.50     4   
1      6000  1350      5            0               3     1.50     4   
2      5286  1800      7            5               4     1.00     4   
3     47916  1590      7            0               3     1.50     3   
4      3825   780      6            0               3     1.00     1   

   bath_full  bath_3qtr  bath_half  golf  greenbelt  noise_traffic  \
0          1          0          0

In [198]:
low_income_df = bin_df[bin_df['price_category'] == 1]
low_income_df = low_income_df.drop(columns=['price_category'])

In [199]:
y = low_income_df["sale_price"].ravel()
scaler3 = StandardScaler()
y_scaled = scaler3.fit_transform(y.reshape(-1, 1)).ravel()
X = low_income_df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)
scaler4 = StandardScaler()
X_scaler = scaler4.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train_scaled, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.5010607400306986
R-squared: 0.5091491390058863


In [201]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [202]:
y_pred2 = rf_model.predict(X_test_scaled)
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler3.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler3.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
rmse = round(rmse, 2)
r2 = r2_score(y_test_original, y_pred2_original)

absolute_diff = np.abs(y_test_original - y_pred2_original)

#Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        879000.00        845659.35             33340.65
1        892500.00        824885.22             67614.78
2        675000.00        700584.86             25584.86
3        940000.00        924506.34             15493.66
4        899999.00        855973.74             44025.26
...            ...              ...                  ...
3054     369000.00        467516.25             98516.25
3055     745000.00        731491.07             13508.93
3056     918000.00        870657.85             47342.15
3057     622500.00        596964.15             25535.85
3058     830000.00        757194.75             72805.25

[3059 rows x 3 columns]
Root Mean Squared Error: 97237.1
R-squared: 0.6602236961719191


In [203]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
y = low_income_df["sale_price"].ravel()
scaler2 = StandardScaler()
y_scaled = scaler2.fit_transform(y.reshape(-1, 1)).ravel()
X = low_income_df.drop("sale_price", axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train a GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, min_samples_split = 10, min_samples_leaf = 5, random_state=42)
gb_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred2 = gb_model.predict(X_test_scaled)

pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler2.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler2.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
rmse = round(rmse, 2)
r2 = r2_score(y_test_original, y_pred2_original) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

#Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        879000.00        943537.87             64537.87
1        892500.00        843567.28             48932.72
2        675000.00        783233.13            108233.13
3        940000.00        887688.78             52311.22
4        899999.00        893442.92              6556.08
...            ...              ...                  ...
3054     369000.00        472114.27            103114.27
3055     745000.00        759528.36             14528.36
3056     918000.00        874475.26             43524.74
3057     622500.00        603481.06             19018.94
3058     830000.00        734844.31             95155.69

[3059 rows x 3 columns]
Root Mean Squared Error: 94005.49
R-squared: 0.6824328980247552


In [193]:
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
groups = bin_df['price_category']  # Replace 'group' with your actual column name

y = bin_df["sale_price"].ravel()
scaler2 = StandardScaler()
y_scaled = scaler2.fit_transform(y.reshape(-1, 1)).ravel()
X = bin_df.drop(["sale_price", "price_category"], axis=1)

# Create a GroupKFold splitter
group_kfold = GroupKFold(n_splits=3)

# Initialize lists to store evaluation metrics
rmse_scores = []
r2_scores = []

# Loop through the splits
for train_index, test_index in group_kfold.split(X, y_scaled, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_scaled[train_index], y_scaled[test_index]

    # Rest of your preprocessing and modeling code
    # ...

    # Calculate evaluation metrics for this fold
    mse = mean_squared_error(y_test_original, y_pred2_original)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, y_pred2_original)

    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Calculate mean of evaluation metrics across all folds
mean_rmse = np.mean(rmse_scores)
mean_r2 = np.mean(r2_scores)

print("Mean Root Mean Squared Error:", mean_rmse)
print("Mean R-squared:", mean_r2)

Mean Root Mean Squared Error: 85712.18596907365
Mean R-squared: 0.6119356987928695
